### Test case LVV-T48: Verify implementation of Exposure Catalog

Verify that the DMS has created an Exposure Catalog containing information for each exposure that includes the exposure date/time and duration, properties of the filter used, dome and telescope pointing and orientation, status of calibration apparatus, airmass and zenith distance, telescope and dome status, environmental information, and information regarding each sensor including an ID, its location in the focal plane, electronic configuration, and WCS.

#### Discussion:

This can be verified using the "ConsDB" (Consolidated database), which is a transformed version of telemetry from the Engineering Facilities Database (EFD), with additional tables and their columns populated by quantities derived from images immediately after they are obtained.

In particular, this test will use on-sky data and telemetry from ComCam observing in late 2024.

In [1]:
# Parameters to select a single observation date and the ComCam instrument:
day_obs = "2024-12-05"
instrument = "lsstcomcam"

Import packages, and set some variables for authentication to ConsDB:

In [2]:
import os
import numpy as np
import pandas as pd
import requests
from IPython.display import Markdown, display, display_markdown
from lsst.summit.utils import ConsDbClient
from astropy.table import Table

URL = "http://consdb-pq.consdb:8080/consdb"

os.environ["no_proxy"] += ",.consdb"

access_token = os.getenv("ACCESS_TOKEN")
headers = {"Authorization": f"Bearer {access_token}"}

%matplotlib inline

Authenticate to and access the ConsDB:

In [3]:
client = ConsDbClient(URL)
print(client)

display_markdown("### Consolidated Database is accessible", raw=True)

### Consolidated Database is accessible

Query the ConsDB over a range of a few days

In [4]:
day_obs_int = int(day_obs.replace("-", ""))
print(f'Date: {day_obs_int}')

# Query the visit table
visit_query1 = f"""
    SELECT * FROM cdb_{instrument}.visit1
    where day_obs <= {day_obs_int} AND day_obs > {day_obs_int-3}
"""

# Print some schema information
try:
    print('\nlist of instruments in client.schema:')
    print(client.schema())  # list the instruments
    print('\nTables in client.schema("lsstcomcam"):')
    print(client.schema("lsstcomcam"))  # list tables for an instrument
except requests.HTTPError or requests.JSONDecodeError:
    print(client.schema())  # list the instruments
    print(client.schema("lsstcomcam"))  # list tables for an instrument

# Execute the query:
try:
    visits_comcam = client.query(visit_query1).to_pandas()
except requests.HTTPError or requests.JSONDecodeError:
    # Try twice
    visits_comcam = client.query(visit_query1).to_pandas()

# Print some stats to the screen:
if len(visits_comcam) > 0:
    print(f"\nRetrieved {len(visits_comcam)} visits from consdb")
    obj_vis = len(visits_comcam.query('img_type == "OBJECT"'))
    print(f"{obj_vis} of these are object images")


Date: 20241205

list of instruments in client.schema:
['latiss', 'lsstcam', 'lsstcamsim', 'lsstcomcam', 'lsstcomcamsim', 'startrackerfast', 'startrackernarrow', 'startrackerwide']

Tables in client.schema("lsstcomcam"):
['cdb_lsstcomcam.exposure', 'cdb_lsstcomcam.ccdexposure', 'cdb_lsstcomcam.ccdexposure_flexdata', 'cdb_lsstcomcam.ccdexposure_flexdata_schema', 'cdb_lsstcomcam.ccdexposure_camera', 'cdb_lsstcomcam.exposure_quicklook', 'cdb_lsstcomcam.exposure_flexdata', 'cdb_lsstcomcam.exposure_flexdata_schema', 'cdb_lsstcomcam.ccdexposure_quicklook', 'cdb_lsstcomcam.ccdvisit1_quicklook', 'cdb_lsstcomcam.visit1_quicklook']

Retrieved 741 visits from consdb
205 of these are object images


#### Print the list of columns in this table:

In [5]:
cols = [col for col in visits_comcam.columns]
print(cols)

['visit_id', 'exposure_name', 'controller', 'day_obs', 'seq_num', 'physical_filter', 'band', 's_ra', 's_dec', 'sky_rotation', 'azimuth_start', 'azimuth_end', 'azimuth', 'altitude_start', 'altitude_end', 'altitude', 'zenith_distance_start', 'zenith_distance_end', 'zenith_distance', 'airmass', 'exp_midpt', 'exp_midpt_mjd', 'obs_start', 'obs_start_mjd', 'obs_end', 'obs_end_mjd', 'exp_time', 'shut_time', 'dark_time', 'group_id', 'cur_index', 'max_index', 'img_type', 'emulated', 'science_program', 'observation_reason', 'target_name', 'air_temp', 'pressure', 'humidity', 'wind_speed', 'wind_dir', 'dimm_seeing', 'focus_z', 'simulated', 's_region']


#### Display statistics related to the date/time and duration of the observation, and the filter:

In [6]:
obs_columns = ['day_obs', 'obs_start', 'obs_end', 'exp_time', 'shut_time',
               'dark_time', 'physical_filter', 'band']

obs_dict = {}

for col in obs_columns:
    obs_dict[col] = visits_comcam.loc[:5][col].values

tab = Table(obs_dict)
tab

day_obs,obs_start,obs_end,exp_time,shut_time,dark_time,physical_filter,band
int64,object,object,float64,float64,float64,object,object
20241203,2024-12-03T19:25:10.904000,2024-12-03T19:25:10.915000,0.0,0.0,0.010877609252929688,r_03,r
20241203,2024-12-03T19:25:13.423000,2024-12-03T19:25:13.431000,0.0,0.0,0.0076389312744140625,r_03,r
20241203,2024-12-03T19:25:15.939000,2024-12-03T19:25:15.948000,0.0,0.0,0.008339166641235352,r_03,r
20241203,2024-12-03T19:25:18.453000,2024-12-03T19:25:18.461000,0.0,0.0,0.008323431015014648,r_03,r
20241203,2024-12-03T19:25:20.967000,2024-12-03T19:25:20.975000,0.0,0.0,0.008099555969238281,r_03,r
20241203,2024-12-03T19:25:23.471000,2024-12-03T19:25:23.477000,0.0,0.0,0.006214141845703125,r_03,r


#### Display quantities related the dome and telescope pointing and orientation:

In [7]:
tel_columns = ['sky_rotation', 'azimuth_start', 'azimuth_end', 'azimuth',
               'altitude_start', 'altitude_end', 'altitude']

tel_dict = {}

for col in tel_columns:
    tel_dict[col] = visits_comcam.loc[:5][col].values

tab = Table(tel_dict)
tab

sky_rotation,azimuth_start,azimuth_end,azimuth,altitude_start,altitude_end,altitude
float64,float64,float64,float64,float64,float64,float64
318.2895316809333,146.286535356456,146.286535005341,146.2865351808985,17.442606373840306,17.442606627817696,17.442606500829
318.28949682118935,146.286535251563,146.286535073797,146.28653516268002,17.442606262725207,17.442606537867405,17.442606400296306
318.28945924115163,146.286535317811,146.286535136733,146.286535227272,17.4426063129916,17.442607114607696,17.44260671379965
318.28942620960134,146.286535010861,146.286535027423,146.286535019142,17.442606918833505,17.442606429397898,17.4426066741157
318.28944324957286,146.286535037361,146.286534946822,146.2865349920915,17.442606701894405,17.4426065881337,17.442606645014052
318.2894098041192,146.286535167649,146.286534858491,146.28653501307,17.442606244206004,17.442607008783796,17.4426066264949


#### Display quantities related to the airmass, zenith distance, and environmental conditions of observations:

In [8]:
env_columns1 = ['zenith_distance_start', 'zenith_distance_end', 'zenith_distance', 'airmass']

env_dict1 = {}

for col in env_columns1:
    env_dict1[col] = visits_comcam.loc[:5][col].values

tab = Table(env_dict1)
tab

zenith_distance_start,zenith_distance_end,zenith_distance,airmass
float64,float64,float64,object
72.5573936261597,72.5573933721823,72.557393499171,None
72.5573937372748,72.5573934621326,72.55739359970369,None
72.5573936870084,72.5573928853923,72.55739328620035,None
72.5573930811665,72.5573935706021,72.5573933258843,None
72.5573932981056,72.5573934118663,72.55739335498595,None
72.557393755794,72.5573929912162,72.5573933735051,None


In [9]:
env_columns2 = ['air_temp', 'pressure', 'humidity', 'wind_speed', 'wind_dir', 'dimm_seeing', 'focus_z']

env_dict2 = {}

for col in env_columns2:
    env_dict2[col] = visits_comcam.loc[:5][col].values

tab = Table(env_dict2)
tab

air_temp,pressure,humidity,wind_speed,wind_dir,dimm_seeing,focus_z
float64,float64,float64,float64,float64,object,float64
11.524999618530273,74135.0,20.049999237060547,9.340800285339355,5.545157432556152,None,0.0
11.524999618530273,74135.0,20.049999237060547,8.923800468444824,9.194999694824219,None,0.0
11.649999618530273,74135.0,19.975000381469727,9.590999603271484,7.809922218322754,None,0.0
11.800000190734863,74135.0,19.862499237060547,8.423399925231934,5.679305553436279,None,0.0
11.800000190734863,74135.0,19.862499237060547,8.089799880981445,11.250134468078613,None,0.0
11.899999618530273,74135.0,19.725000381469727,7.422599792480469,10.949991226196289,None,0.0


In [10]:
# Query the ccdexposure table
ccdexp_query = f"""
    SELECT * FROM cdb_{instrument}.ccdexposure
    where day_obs <= {day_obs_int} AND day_obs > {day_obs_int-3}
"""

# Execute the query:
try:
    ccdexp_comcam = client.query(ccdexp_query).to_pandas()
except requests.HTTPError or requests.JSONDecodeError:
    # Try twice
    ccdexp_comcam = client.query(visit_query).to_pandas()


In [11]:
ccdexp_comcam

,ccdexposure_id,exposure_id,detector,s_region,day_obs,seq_num
0,45717913856,2024120300001,0,None,20241203,1
1,45717913857,2024120300001,1,None,20241203,1
2,45717913858,2024120300001,2,None,20241203,1
3,45717913859,2024120300001,3,None,20241203,1
4,45717913860,2024120300001,4,None,20241203,1
...,...,...,...,...,...,...
6664,45734769668,2024120500308,4,None,20241205,308
6665,45734769669,2024120500308,5,None,20241205,308
6666,45734769670,2024120500308,6,None,20241205,308
6667,45734769671,2024120500308,7,None,20241205,308


## Results

We see that the ConsDB contains information taken directly from image headers, transformed data from the EFD, and derived data based on image processing. As required, these data include information about each exposure, include the telescope and instrument configuration, telemetry from the telescope, environmental and pointing information, and details about the camera.

In some examples shown above, columns that exist in the database are not yet populated. Nonetheless, this test is deemed to *PASS* because we have demonstrated the existence of a database (ConsDB) keyed on day_obs, which can take inputs from a variety of sources, all of which can be configured.